In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
import scipy.signal as ss
import scipy

In [2]:
import electrode_tester

In [7]:
def generate_sine(amplitude, frequency, phase, offset, noise, fs, T):
    t = np.arange(0, T, 1/fs)
    s = amplitude * np.sin(t * 2 * np.pi * frequency + phase) + offset + np.random.uniform(0, noise) * (2 * np.random.random(t.size) - 1)
    return t, s


In [ ]:
ranges = {
    'amplitude': (0.1, 4),
    'phase': (0, 2 * np.pi),
    'offset': (-1, 1) #(-1000, 1000),
}

freqs = np.arange(1, 101)
TEST_N = 1000
params = ['amplitude', 'phase', 'offset']

methods = ['Fourier', 'Hilbert', 'FitSin']
e1 = electrode_tester.estimators.FitFourier()
e2 = electrode_tester.estimators.FitHilbert()
e3 = electrode_tester.estimators.FitFourier()
fun = [e1.estimate, e2.estimate, e3.estimate]

errors_matrix = np.zeros((len(methods), len(freqs), len(params), TEST_N))

for f, freq in enumerate(freqs):
    start = time.time()
    
    for i in range(TEST_N):
        true_params = {}
        for param in params:
            true_params[param] = np.random.uniform(*ranges[param])
        
        noise = 0.05
        fs = 1000
        T = 5
        t, s = generate_sine(true_params['amplitude'], freq, true_params['phase'], true_params['offset'], noise, fs, T)
        
        estimator_params = {
        'frequency': freq, 
        'fs': fs
        }
        
        for m, method in enumerate(methods):
            estimated_params = fun[m](np.copy(t), np.copy(s), **estimator_params)

            errors = {}
            for p, param in enumerate(params):

                true_value = true_params[param]
                estimated_value = estimated_params._asdict()[param]
                if param == 'phase' and ((np.round(true_value, 2) == 0 and np.round(estimated_value, 2) == 6.28) or (np.round(true_value, 2) == 6.28 and np.round(estimated_value, 2) == 0)):
                    true_params[param] = 0
                    estimated_params._replace(phase=0)
                error = (true_params[param] - estimated_params._asdict()[param])

                errors_matrix[m, f, p, i] = error
                if error > 0.1:
                    expected = true_params[param]
                    calcualted = estimated_params._asdict()[param]
                    print(f'{method} {param}: {error=}, {expected=}, {calcualted=}')
            
    print(f'Freq {freq}Hz: {(time.time() - start) / 60:.4f} min')


Freq 1Hz: 0.0418 min
Freq 2Hz: 0.0428 min
Freq 3Hz: 0.0405 min
Freq 4Hz: 0.0445 min
Freq 5Hz: 0.0441 min
Freq 6Hz: 0.0424 min
Freq 7Hz: 0.0408 min
Freq 8Hz: 0.0415 min
Freq 9Hz: 0.0425 min
Freq 10Hz: 0.0415 min
Freq 11Hz: 0.0421 min
Freq 12Hz: 0.0407 min
Freq 13Hz: 0.0439 min
Freq 14Hz: 0.0419 min
Freq 15Hz: 0.0410 min
Freq 16Hz: 0.0426 min
Freq 17Hz: 0.0440 min
Freq 18Hz: 0.0421 min
Freq 19Hz: 0.0413 min
Freq 20Hz: 0.0412 min
Freq 21Hz: 0.0404 min
Freq 22Hz: 0.0413 min
Freq 23Hz: 0.0405 min
Freq 24Hz: 0.0425 min
Freq 25Hz: 0.0410 min
Freq 26Hz: 0.0415 min
Freq 27Hz: 0.0420 min
Freq 28Hz: 0.0429 min
Freq 29Hz: 0.0404 min
Freq 30Hz: 0.0401 min
Freq 31Hz: 0.0404 min
Freq 32Hz: 0.0406 min
Freq 33Hz: 0.0414 min
Freq 34Hz: 0.0415 min
Freq 35Hz: 0.0406 min
Freq 36Hz: 0.0404 min
Freq 37Hz: 0.0438 min
Freq 38Hz: 0.0420 min
Freq 39Hz: 0.0405 min
Freq 40Hz: 0.0404 min
Freq 41Hz: 0.0405 min
Freq 42Hz: 0.0402 min
Freq 43Hz: 0.0401 min
Freq 44Hz: 0.0403 min
Freq 45Hz: 0.0405 min
Freq 46Hz: 0.0404 m

In [ ]:
#np.save("errors.npy", E)
std_matrix = np.std(errors_matrix, axis=-1)
sc = 0.2
plt.figure(figsize = np.array([15, 15]))
units = ['V', 'rad', 'V']
params = ['amplitudy', 'fazy', 'offsetu']
methods_names = ['Fourier', 'Sygnał analityczny', 'FitSin']
plot_idx = 1
for p, param in enumerate(params):
    for m, method in enumerate(methods_names):
        param_std = np.mean(std_matrix[m, :, p])
        print(f'{method} {param}: {param_std}')
        plt.subplot(len(methods_names), len(methods_names), plot_idx)
        plt.plot(freqs, std_matrix[m, :, p], 'r.')
        plt.ylim(0, 1.1 * np.max(std_matrix[:, :, p]))
        if plot_idx < len(metody) + 1:
            plt.title(method, fontsize = sc*60)
        if plot_idx % len(methods_names) == 1:
            plt.ylabel("Odchylenie standardowe błędu dopasowania\ndla " + params[p] + " [" + units[p] + "]", fontsize = sc*60)
        if plot_idx > (len(params) - 1) * len(methods_names):
            plt.xlabel("Częstotliwość [Hz]", fontsize = sc*60)
        plot_idx += 1
plt.savefig("Errors.png")
plt.show()